** Created by Vaibhav Singh **

In [ ]:
!pwd

In [2]:
!pip3 install git+https://github.com/huggingface/datasets

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/huggingface/datasets to /tmp/pip-req-build-478ti7df
  Running command git clone -q https://github.com/huggingface/datasets /tmp/pip-req-build-478ti7df
     |████████████████████████████████| 76 kB 9.4 MB/s  eta 0:00:01
     |████████████████████████████████| 1.3 MB 82.7 MB/s eta 0:00:01
     |████████████████████████████████| 50 kB 13.0 MB/s eta 0:00:01
     |████████████████████████████████| 293 kB 103.8 MB/s eta 0:00:01
     |████████████████████████████████| 141 kB 102.3 MB/s eta 0:00:01
  Created wheel for datasets: filename=datasets-1.11.1.dev0-py3-none-any.whl size=267131 sha256=fa310f887813172a65e66fc86b13ef66b823021e7be563933c5699648b8a5527
  Stored in directory: /tmp/pip-ephem-wheel-cache-aa5n83_r/wheels/05/48/ba/616197d38711416979c8a15a5137a436d1d421f7f905eb3d1b
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-py3-none-any.whl size=3178 sha256=014b6efe55636a650f2285

In [3]:
!ls

Untitled.ipynb


In [4]:
#!pip3 install datasets==1.4.1
!pip3 install transformers==4.4.0
!pip3 install soundfile
!pip3 install jiwer

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [5]:
from datasets import load_dataset, load_metric

timit = load_dataset("timit_asr")

print(timit)

Downloading:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.06k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset timit_asr downloaded and prepared to /home/ubuntu/.cache/huggingface/datasets/timit_asr/clean/2.0.1/66d672b7070257726bc9e76100d06d3e342aa6c37aed742803302c293540e96d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['file', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 4620
    })
    test: Dataset({
        features: ['file', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 1680
    })
})


In [ ]:
timit = timit.remove_columns(["phonetic_detail", "word_detail", "dialect_region", "id", "sentence_type", "speaker_id"])

In [8]:
from datasets import ClassLabel
import random
import pandas as pd
from IPython.display import display, HTML

# Random samples from dataset

In [9]:
def show_random_elements(dataset, num_examples=10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    display(HTML(df.to_html()))

In [10]:
show_random_elements(timit["train"].remove_columns(["file"]))

,text
0,The lack of heat compounded the tenant's grievances.
1,How would you evaluate this algebraic expression?
2,The tapes show that.
3,Fill that canteen with fresh spring water.
4,"It does not indicate loose management, ineffective controls or poor policy."
5,Don't ask me to carry an oily rag like that.
6,Be careful not to plow over the flower beds.
7,Don't ask me to carry an oily rag like that.
8,Toothpaste tube should be squeezed from the bottom.
9,Like as if it were built of books.


In [12]:
import re
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'

def remove_special_characters(batch):
    batch["text"] = re.sub(chars_to_ignore_regex, '', batch["text"]).lower()
    return batch

timit = timit.map(remove_special_characters)

  0%|          | 0/4620 [00:00<?, ?ex/s]

  0%|          | 0/1680 [00:00<?, ?ex/s]

In [13]:
show_random_elements(timit["train"].remove_columns(["file"]))

,text
0,don't ask me to carry an oily rag like that
1,it had gone like clockwork
2,don't ask me to carry an oily rag like that
3,thyroid function tests yielded normal results
4,a leather handbag would be a suitable gift
5,his fingers felt absurdly thick and clumsy
6,i'll have a scoop of that exotic purple and turquoise sherbet
7,again these blocks were set in resinsaturated glass cloth and nailed
8,she had your dark suit in greasy wash water all year
9,don't they still call you junior as though you're about ten years old


In [14]:
def extract_all_chars(batch):
  all_text = " ".join(batch["text"])
  vocab = list(set(all_text))
  return {"vocab": [vocab], "all_text": [all_text]}

In [15]:
vocabs = timit.map(extract_all_chars, batched=True, batch_size=-1, keep_in_memory=True, remove_columns=timit.column_names["train"])

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [16]:
vocab_list = list(set(vocabs["train"]["vocab"][0]) | set(vocabs["test"]["vocab"][0]))

vocab_dict = {v: k for k, v in enumerate(vocab_list)}
vocab_dict

{'p': 0,
 'y': 1,
 'q': 2,
 'f': 3,
 'a': 4,
 's': 5,
 'm': 6,
 'o': 7,
 'x': 8,
 'g': 9,
 'c': 10,
 't': 11,
 "'": 12,
 'h': 13,
 'w': 14,
 'e': 15,
 'l': 16,
 ' ': 17,
 'b': 18,
 'i': 19,
 'z': 20,
 'r': 21,
 'd': 22,
 'u': 23,
 'j': 24,
 'k': 25,
 'v': 26,
 'n': 27}

In [17]:
vocab_dict["|"] = vocab_dict[" "]
del vocab_dict[" "]

In [18]:
vocab_dict["[UNK]"] = len(vocab_dict)
vocab_dict["[PAD]"] = len(vocab_dict)
print(len(vocab_dict))

30


In [19]:
import json
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)

# Tokenizer and Feature extractor

In [20]:
from transformers import Wav2Vec2CTCTokenizer

In [21]:
tokenizer = Wav2Vec2CTCTokenizer("./vocab.json", unk_token="[UNK]", pad_token="[PAD]", word_delimiter_token="|")

In [22]:
from transformers import Wav2Vec2FeatureExtractor

In [23]:
feature_extractor = Wav2Vec2FeatureExtractor(feature_size=1, sampling_rate=16000, padding_value=0.0, do_normalize=True, return_attention_mask=False)

In [24]:
from transformers import Wav2Vec2Processor

In [25]:
processor = Wav2Vec2Processor(feature_extractor=feature_extractor, tokenizer=tokenizer)

In [27]:
print(timit["train"][1])

{'file': '/home/ubuntu/.cache/huggingface/datasets/downloads/extracted/75045eb679e4518fc6a8f0aee618f5740e046ecca85c45da5bf6a16075214e8a/data/TRAIN/DR4/MMDM0/SA2.WAV', 'text': "don't ask me to carry an oily rag like that"}


# Spectogram

In [28]:
import soundfile as sf

def speech_file_to_array_fn(batch):
    speech_array, sampling_rate = sf.read(batch["file"])
    batch["speech"] = speech_array
    batch["sampling_rate"] = sampling_rate
    batch["target_text"] = batch["text"]
    return batch

In [29]:
timit = timit.map(speech_file_to_array_fn, remove_columns=timit.column_names["train"], num_proc=4)

In [30]:
import IPython.display as ipd
import numpy as np
import random

In [34]:
rand_int = random.randint(0, len(timit["train"]))

ipd.Audio(data=np.asarray(timit["train"][rand_int]["speech"]), autoplay=True, rate=16000)

In [35]:
rand_int = random.randint(0, len(timit["train"]))

print("Target text:", timit["train"][rand_int]["target_text"])
print("Input array shape:", np.asarray(timit["train"][rand_int]["speech"]).shape)
print("Sampling rate:", timit["train"][rand_int]["sampling_rate"])

Target text: the cow wandered from the farmland and became lost
Input array shape: (40960,)
Sampling rate: 16000


# Data Prepearation

In [36]:
def prepare_dataset(batch):
    # check that all files have the correct sampling rate
    assert (
        len(set(batch["sampling_rate"])) == 1
    ), f"Make sure all inputs have the same sampling rate of {processor.feature_extractor.sampling_rate}."

    batch["input_values"] = processor(batch["speech"], sampling_rate=batch["sampling_rate"][0]).input_values

    with processor.as_target_processor():
        batch["labels"] = processor(batch["target_text"]).input_ids
    return batch

In [37]:
timit_prepared = timit.map(prepare_dataset, remove_columns=timit.column_names["train"], batch_size=8, num_proc=4, batched=True)

/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)
/usr/local/lib/python3.6/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is de

In [38]:
import torch

from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union

In [39]:
@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lenghts and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

In [40]:
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)

In [41]:
wer_metric = load_metric("wer")

Downloading:   0%|          | 0.00/1.95k [00:00<?, ?B/s]

# Metrics for evaluvation

In [42]:
def compute_metrics(pred):
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id

    pred_str = processor.batch_decode(pred_ids)
    # we do not want to group tokens when computing the metrics
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)

    wer = wer_metric.compute(predictions=pred_str, references=label_str)

    return {"wer": wer}

# Model Building

In [43]:
from transformers import Wav2Vec2ForCTC

In [44]:
model = Wav2Vec2ForCTC.from_pretrained(
    "facebook/wav2vec2-base", 
    gradient_checkpointing=True, 
    ctc_loss_reduction="mean", 
    pad_token_id=processor.tokenizer.pad_token_id,
)

Downloading:   0%|          | 0.00/1.84k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/380M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/wav2vec2-base were not used when initializing Wav2Vec2ForCTC: ['quantizer.codevectors', 'quantizer.weight_proj.weight', 'quantizer.weight_proj.bias', 'project_q.weight', 'project_q.bias', 'project_hid.weight', 'project_hid.bias']
- This IS expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing Wav2Vec2ForCTC from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base and are newly initialized: ['lm_head.weight', 'lm_head.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [45]:
model.freeze_feature_extractor()

In [46]:
from transformers import TrainingArguments

# Training parameters

In [47]:
training_args = TrainingArguments(
  # output_dir="/content/gdrive/MyDrive/wav2vec2-base-timit-demo",
  output_dir="./wav2vec2-base-timit-demo",
  group_by_length=True,
  per_device_train_batch_size=32,
  evaluation_strategy="steps",
  num_train_epochs=30,
  fp16=True,
  save_steps=500,
  eval_steps=500,
  logging_steps=500,
  learning_rate=1e-4,
  weight_decay=0.005,
  warmup_steps=1000,
  save_total_limit=2,
)

In [48]:
from transformers import Trainer

In [49]:
trainer = Trainer(
    model=model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=timit_prepared["train"],
    eval_dataset=timit_prepared["test"],
    tokenizer=processor.feature_extractor,
)

# Training

In [50]:
trainer.train()

/home/ubuntu/.local/lib/python3.6/site-packages/torch/optim/lr_scheduler.py:134: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Step,Training Loss,Validation Loss,Wer,Runtime,Samples Per Second
500,3.774700,2.603045,1.000000,95.935800,17.512000
1000,0.813000,0.434351,0.373165,97.569100,17.219000
1500,0.196200,0.404099,0.328440,98.148100,17.117000
2000,0.112500,0.410039,0.299635,98.359200,17.080000
2500,0.074500,0.431421,0.291985,98.783800,17.007000
3000,0.056900,0.444837,0.284267,98.535600,17.050000
3500,0.047000,0.427965,0.283854,98.637300,17.032000
4000,0.034400,0.429865,0.279926,98.709400,17.020000


TrainOutput(global_step=4350, training_loss=0.5896289419853824, metrics={'train_runtime': 8042.5395, 'train_samples_per_second': 0.541, 'total_flos': 3.9858324576556134e+18, 'epoch': 30.0, 'init_mem_cpu_alloc_delta': 63727, 'init_mem_gpu_alloc_delta': 377847808, 'init_mem_cpu_peaked_delta': 18282, 'init_mem_gpu_peaked_delta': 0, 'train_mem_cpu_alloc_delta': 1073050, 'train_mem_gpu_alloc_delta': 1118267392, 'train_mem_cpu_peaked_delta': 127231827, 'train_mem_gpu_peaked_delta': 4099869184})

In [51]:
processor = Wav2Vec2Processor.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo")
model = Wav2Vec2ForCTC.from_pretrained("patrickvonplaten/wav2vec2-base-timit-demo")

Downloading:   0%|          | 0.00/159 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/268 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/138 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


Downloading:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/378M [00:00<?, ?B/s]

# Model output to readable format

In [52]:
def map_to_result(batch):
  model.to("cuda")
  input_values = processor(
      batch["speech"], 
      sampling_rate=batch["sampling_rate"], 
      return_tensors="pt"
  ).input_values.to("cuda")

  with torch.no_grad():
    logits = model(input_values).logits

  pred_ids = torch.argmax(logits, dim=-1)
  batch["pred_str"] = processor.batch_decode(pred_ids)[0]

  return batch


# Random results from the test set

In [53]:
results = timit["test"].map(map_to_result)


  0%|          | 0/1680 [00:00<?, ?ex/s]

In [54]:
print("Test WER: {:.3f}".format(wer_metric.compute(predictions=results["pred_str"], references=results["target_text"])))


Test WER: 0.186


In [55]:
show_random_elements(results.remove_columns(["speech", "sampling_rate"]))


,target_text,pred_str
0,aim to balance your employee benefit package,am to balence your employe you benefits package
1,the fog prevented them from arriving on time,the fawlg prevented them from ariving on tom
2,young children should avoid exposure to contagious diseases,young children should avoide exposure to contagieous diseases
3,artificial intelligence is for real,artifficial intelligence is for real
4,their props were two stepladders a chair and a palm fan,their pcrops were two step latters a chair and a polmb fan
5,if people were more generous there would be no need for welfare,if people were more generous there would be no need for wealfare
6,the fish began to leap frantically on the surface of the small lake,the fish began to leep frantically on the surface of the small ac
7,her right hand aches whenever the barometric pressure changes,her right hand eggs whenever the barametric pressur changes
8,only lawyers love millionaires,only lawyers loved miliunears
9,the nearest synagogue may not be within walking distance,the nearest cennagade may not be within wallkin distance


In [56]:
model.to("cuda")
input_values = processor(timit["test"][0]["speech"], sampling_rate=timit["test"][0]["sampling_rate"], return_tensors="pt").input_values.to("cuda")

with torch.no_grad():
  logits = model(input_values).logits

pred_ids = torch.argmax(logits, dim=-1)

# convert ids to tokens
print(" ".join(processor.tokenizer.convert_ids_to_tokens(pred_ids[0].tolist())))

[PAD] [PAD] [PAD] [PAD] [PAD] [PAD] t t h e e | | b b [PAD] u u n n n g g [PAD] a [PAD] [PAD] l l [PAD] o o o [PAD] | w w a a [PAD] s s | | [PAD] [PAD] p l l e e [PAD] [PAD] s s e n n t t t [PAD] l l y y | | | s s [PAD] i i [PAD] t t t [PAD] u u u u [PAD] [PAD] [PAD] a a [PAD] t t e e e d d d | n n e e a a a r | | t h h e | | s s h h h [PAD] o o o [PAD] o o r r [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD] [PAD]


# Model Summary

In [57]:
model

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureExtractor(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (2): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (3): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (4): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (5): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
        )
        (6): Wav2Vec

In [62]:
rand_int = random.randint(0, len(timit["train"]))

ipd.Audio(data=np.asarray(timit["train"][rand_int]["speech"]), autoplay=True, rate=16000)

In [64]:
timit["train"][rand_int]['target_text']

'who took the kayak down the bayou'

In [65]:
rand_int = random.randint(0, len(timit["train"]))

ipd.Audio(data=np.asarray(timit["train"][rand_int]["speech"]), autoplay=True, rate=16000)

Dataset({
    features: ['speech', 'sampling_rate', 'target_text'],
    num_rows: 4620
})

In [66]:
timit["train"][rand_int]["speech"]

[-0.001800537109375,
 0.00067138671875,
 -0.000274658203125,
 0.000244140625,
 -6.103515625e-05,
 6.103515625e-05,
 9.1552734375e-05,
 6.103515625e-05,
 3.0517578125e-05,
 0.000152587890625,
 0.000152587890625,
 9.1552734375e-05,
 6.103515625e-05,
 3.0517578125e-05,
 6.103515625e-05,
 0.0,
 3.0517578125e-05,
 3.0517578125e-05,
 0.0001220703125,
 0.0,
 6.103515625e-05,
 -9.1552734375e-05,
 6.103515625e-05,
 0.0001220703125,
 0.0001220703125,
 -0.0001220703125,
 6.103515625e-05,
 3.0517578125e-05,
 -9.1552734375e-05,
 9.1552734375e-05,
 0.0,
 6.103515625e-05,
 3.0517578125e-05,
 -9.1552734375e-05,
 0.0,
 -6.103515625e-05,
 3.0517578125e-05,
 -0.0001220703125,
 -0.0001220703125,
 -0.0001220703125,
 0.0,
 -9.1552734375e-05,
 -3.0517578125e-05,
 0.0,
 -9.1552734375e-05,
 -0.0001220703125,
 -3.0517578125e-05,
 0.0,
 -9.1552734375e-05,
 -3.0517578125e-05,
 6.103515625e-05,
 0.0,
 0.0,
 -3.0517578125e-05,
 9.1552734375e-05,
 6.103515625e-05,
 0.0001220703125,
 6.103515625e-05,
 0.0,
 6.1035156

In [87]:
check_timit = load_dataset("timit_asr")

Reusing dataset timit_asr (/home/ubuntu/.cache/huggingface/datasets/timit_asr/clean/2.0.1/66d672b7070257726bc9e76100d06d3e342aa6c37aed742803302c293540e96d)


  0%|          | 0/2 [00:00<?, ?it/s]

In [88]:
check_timit

DatasetDict({
    train: Dataset({
        features: ['file', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 4620
    })
    test: Dataset({
        features: ['file', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
        num_rows: 1680
    })
})

# Test set

In [89]:
check_timit["test"]

Dataset({
    features: ['file', 'text', 'phonetic_detail', 'word_detail', 'dialect_region', 'sentence_type', 'speaker_id', 'id'],
    num_rows: 1680
})

In [92]:
check_timit["test"]["file"][0]

'/home/ubuntu/.cache/huggingface/datasets/downloads/extracted/75045eb679e4518fc6a8f0aee618f5740e046ecca85c45da5bf6a16075214e8a/data/TEST/DR4/MGMM0/SX139.WAV'

In [98]:
test_speech_array, test_sampling_rate = sf.read(check_timit["test"]["file"][0])
test_batch["speech"] = test_speech_array
test_batch["sampling_rate"] = test_sampling_rate
test_batch["target_text"] = check_timit["test"]["text"][0]

In [101]:
ipd.Audio(data=np.asarray(test_batch["speech"]), autoplay=True, rate=16000)


In [102]:
print(test_batch["target_text"])

The bungalow was pleasantly situated near the shore.


In [106]:
model.to("cuda")

Wav2Vec2ForCTC(
  (wav2vec2): Wav2Vec2Model(
    (feature_extractor): Wav2Vec2FeatureExtractor(
      (conv_layers): ModuleList(
        (0): Wav2Vec2GroupNormConvLayer(
          (conv): Conv1d(1, 512, kernel_size=(10,), stride=(5,), bias=False)
          (layer_norm): GroupNorm(512, 512, eps=1e-05, affine=True)
        )
        (1): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (2): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (3): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (4): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(3,), stride=(2,), bias=False)
        )
        (5): Wav2Vec2NoLayerNormConvLayer(
          (conv): Conv1d(512, 512, kernel_size=(2,), stride=(2,), bias=False)
        )
        (6): Wav2Vec

In [114]:
test_input = processor(test_batch["speech"], sampling_rate=test_batch["sampling_rate"]).input_values

In [117]:
len(test_input[0])

44749

In [118]:
logits = model(input_values).logits

In [119]:
pred_ids = torch.argmax(logits, dim=-1)

In [120]:
test_batch["pred_str"] = processor.batch_decode(pred_ids)[0]

In [121]:
test_batch["pred_str"]

'the bungalo was plesently situated near the shoor'

# READING AUDIO FILE AND PREDICTING ITS CORRESPONDING TEXT

In [135]:
!ls

test_audio.wav	transcribe.ipynb  vocab.json  wav2vec2-base-timit-demo


In [144]:
data,sample_rate= sf.read("test_audio.wav")

In [145]:
print(sample_rate)
len(data)

16000


244736

In [139]:
ipd.Audio(data=np.asarray(data), autoplay=True, rate=16000)

### Inference function 

In [150]:
def predict(filename):
    data,sample_rate= sf.read(filename)
    print("Sampling rate :", sample_rate)
    print("Data length :", len(data))
    #input_values = processor(data, sampling_rate=sample_rate).input_values
    input_values = processor(data, sampling_rate=sample_rate, return_tensors="pt").input_values.to("cuda")
    print(input_values.shape)
    logits = model(input_values).logits
    pred_ids = torch.argmax(logits, dim=-1)
    pred_str = processor.batch_decode(pred_ids)[0]
    print(pred_str)

In [151]:
predict("test_audio.wav")

Sampling rate : 16000
Data length : 244736
torch.Size([1, 244736])
helomyi namem iss balos theme i m romglook knoni uggather ombredobresanin juniaen had dubad i hestoy doub neame dis ou do


# Post processing

No model will give 100% accuracy, so in order to get production ready we need to put autocorrect tool in the pipeline. For auto-correct we can do following things :

* use **GingerIt**, opensource python project for autocorrect
* existing python libraries : spellchecker, JamSpell (language based model), textblob, etc. 
* one can train own model using some transfer learning techniques with BERT as base model

### Note : The sampling rate of dataset should match with what is used in real scenario.

## Future Work

* Get real indian dataset of audio clips and corresponding texts. No need of mapping those texts to each segment to audio as this
model is capable of handling those things.
* Big gpu for training the model
* Test using above inference function and caluclate metrics : word rate error
* Post processing